### This notebook includes annual data for:  
+ Unemployment Rate  
+ GDP  
+ Educational Awards  
+ Occupational Employment  
+ Industry Employment at the Census Place level  

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from functools import reduce
pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y
def percent(x, y):
    return (x/y)*100

Unemployment Rate:  

To retrieve data:  
+ Log into JobsEQ and navigate to the "Data Explorer" function  
+ go to "Draft Mode"
+ Select the "LAUS" dataset, annual
+ Select the following dimensions: Region, Year 
+ Under "Region" select "USA", "Tennessee", GNRC (13), MPO (7), GNRC Region (14), and then under GNRC Region sub all "Counties" (not available for places)     
+ Under "Year" choose your range (most recently I've done 2000 - recent)
+ Select the value "Unemployment Rate".  

When you've downloaded this csv, open it and replace the title row with the year (as all of the values are unemployment rates), delete the header line ("LAUS") and data notes at the bottom, rename region to "NAME". You can also quickly delimit the NAMEs to remove the ", Tennessee" off of the end of the counties and change "USA" to "US". This can also be done here, of course. Select all numerical data and change format to "general". Save as 'JobsEQ_LAUSAnnual_GeosExceptPlaces.csv'

In [220]:
#import LAUS data and examine
data = pd.read_csv('../../Data Downloads/JobsEQ_LAUSAnnual_GeosExceptPlaces.csv')
data.head(2)

,NAME,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,US,0.040500,0.047934,0.058466,0.060384,0.055739,0.051692,0.046853,0.046817,0.058406,0.093022,0.096888,0.090063,0.081186,0.074261,0.062230,0.053304,0.049170,0.043990,0.039305,0.036993,0.080837,0.053713,0.036654
1,Tennessee,0.038694,0.045155,0.051234,0.055160,0.053448,0.055293,0.052631,0.046434,0.064913,0.101404,0.098303,0.091168,0.079641,0.077549,0.066052,0.055797,0.047417,0.037384,0.034908,0.033986,0.073829,0.042880,0.034079


In [221]:
#make a list of the columns to melt from wide to long format with the year as a variable and naming the value "unemployment %"
#it's important to leave the percent because in all of the data pipeline percentages have this character in the string title so they can be reformatted if necessary
cols = data.columns
data = data.melt(var_name = 'Year', id_vars = ['NAME'], value_vars = cols, value_name = 'LF:Unemployment %')

In [222]:
#multiply this by 100 to be consistent with other formatting
data['LF:Unemployment %'] = data['LF:Unemployment %'] * 100

In [223]:
ue = data
ue.head(2)

,NAME,Year,LF:Unemployment %
0,US,2000,4.049995
1,Tennessee,2000,3.869428


In [224]:
ue['Year'] = ue['Year'].astype(str)

GDP:  

To retrieve data:  
+ Log into JobsEQ and navigate to the "Data Explorer" function  
+ go to "Draft Mode"
+ Select the "GDP" dataset  
+ Select the following dimensions: Region, Year 
+ Under "Region" select "USA", "Tennessee", GNRC (13), MPO (7), GNRC Region (14), and then under GNRC Region sub all "Counties" (not available for places)     
+ Under "Year" choose your range (most recently I've selected the full available range of 2001 - recent **Watch out here - keep old year if range changes and add onto new download**)  
+ Select the value "GDP".  

When you've downloaded this csv, open it and replace the title row with the year (as all of the values are GDPs), delete the header line and data notes at the bottom, rename region to "NAME", and select all of the numerical data and make them "General" format. You can also quickly delimit the NAMEs to remove the ", Tennessee" off of the end of the counties and change "USA" to "US". This can also be done here, of course. Save as 'JobsEQ_GDPAnnual_GeosExceptPlaces.csv'

In [192]:
#GDP import data and examine
data = pd.read_csv('../../Data Downloads/JobsEQ_GDPAnnual_GeosExceptPlaces.csv')
data.head(2)

,NAME,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,US,1.058190e+13,1.092910e+13,1.145640e+13,1.221720e+13,1.303920e+13,1.381560e+13,1.447420e+13,1.476980e+13,1.447810e+13,1.504900e+13,1.559970e+13,1.625400e+13,1.684320e+13,1.755070e+13,1.820570e+13,1.869490e+13,1.947960e+13,2.052680e+13,2.137250e+13,2.089370e+13,2.299610e+13
1,Tennessee,1.864350e+11,1.957600e+11,2.051200e+11,2.193220e+11,2.316270e+11,2.435780e+11,2.486910e+11,2.542420e+11,2.523330e+11,2.577760e+11,2.698140e+11,2.863420e+11,2.958630e+11,3.061530e+11,3.252940e+11,3.364140e+11,3.498380e+11,3.613800e+11,3.769150e+11,3.695720e+11,4.182940e+11


In [193]:
#make a list of the columns to melt from wide to long format with the year as a variable and naming the value "GDP"
cols = data.columns
data = data.melt(var_name = 'Year', id_vars = ['NAME'], value_vars = cols, value_name = 'GDP:GDP')

In [194]:
#final check
data.head(2)

,NAME,Year,GDP:GDP
0,US,2001,1.058190e+13
1,Tennessee,2001,1.864350e+11


In [195]:
#make a dataframe called gdp and ensure that the year is a string to keep the dfs straight and join later
gdp = data
gdp['Year'] = gdp['Year'].astype(str)

Awards:  

To retrieve data:  
+ Log into JobsEQ and navigate to the "Data Explorer" function  
+ go to "Draft Mode"
+ Select the "Education" dataset  
+ Select the following dimensions: Region and Schools, Year, and Award Level   
+ Under "Region" select "GNRC Region - All Schools", and then sub counties    
+ Under "Year" choose your range (most recently I've selected the full available range of 2010 - recent **Watch out here - keep old year if range changes and add onto new download**)  
+ Under Award Level leave "All Award Levels", and then all subs  
+ Select the value "Awards"  
+ Swap rows & columns so the "Regions & Schools" data is at the top, "Award Level" is on the side, as well as "Year"  

When you've downloaded this csv, open it and replace the title row with the geography (as all of the values are all awards) while keeping the headers for award level and year, then delete the top line and data notes at the bottom, and select all of the numerical data and make them "General" format. Save as 'JobsEQ_AwardsAnnual_GeosExceptPlaces.csv'

In [237]:
#Awards import data and examine
data = pd.read_csv('../../Data Downloads/JobsEQ_AwardsAnnual_GeosExceptPlaces.csv')
data.head(2)

,Award Level,Year,GNRC Region-All Schools,"Cheatham County, Tennessee-All Schools (47021)","Davidson County, Tennessee-All Schools (47037)","Dickson County, Tennessee-All Schools (47043)","Houston County, Tennessee-All Schools (47083)","Humphreys County, Tennessee-All Schools (47085)","Maury County, Tennessee-All Schools (47119)","Montgomery County, Tennessee-All Schools (47125)","Robertson County, Tennessee-All Schools (47147)","Rutherford County, Tennessee-All Schools (47149)","Stewart County, Tennessee-All Schools (47161)","Sumner County, Tennessee-All Schools (47165)","Trousdale County, Tennessee-All Schools (47169)","Williamson County, Tennessee-All Schools (47187)","Wilson County, Tennessee-All Schools (47189)",Unnamed: 17
0,All Award Levels,2010,25779,NaN,15864,294.0,NaN,NaN,699.0,2166.0,NaN,4891,NaN,1033.0,242.0,55.0,535.0,NaN
1,All Award Levels,2011,28038,NaN,17182,394.0,NaN,NaN,676.0,2517.0,NaN,5338,NaN,1033.0,249.0,96.0,553.0,NaN


In [238]:
#don't know where this came from, can revisit later... but drop weird column
data.drop(columns = 'Unnamed: 17', inplace = True)

In [239]:
#make a list of columns to take from wide to long format naming the column of current columns "NAME" and the value "Awards"
cols = data.columns
data = data.melt(var_name = 'NAME', id_vars = ['Award Level', 'Year'], value_vars = cols, value_name = 'Awards')
data.head(2)

,Award Level,Year,NAME,Awards
0,All Award Levels,2010,GNRC Region-All Schools,25779.0
1,All Award Levels,2011,GNRC Region-All Schools,28038.0


In [240]:
#split the NAME on common delimiters and expand the split into a new dataframe, where you'll take the first instance and write-over the original "NAME" and strip
boop = data['NAME'].str.split(r"\-|,", expand = True)
data['NAME'] = boop[0].str.strip()

In [241]:
#make lists of years and geographies to loop through and group the different award levels
years = list(data['Year'].unique())
geos = list(data['NAME'].unique())

In [242]:
#format the column headers so that each individual instance can be looped through, and you can convert to float types before by dropping null then converting
data = data.set_index(['Award Level', 'Year', 'NAME']).transpose()
data.head()

Award Level All Award Levels                                                                                                              \
Year                    2010        2011        2012        2013        2014        2015        2016        2017        2018        2019   
NAME             GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region   
Awards               25779.0     28038.0     30017.0     28404.0     28353.0     28549.0     28580.0     28689.0     27726.0     27785.0   

Award Level                         Certificate < 1 Yr                                                                                      \
Year               2020        2021               2010        2011        2012        2013        2014        2015        2016        2017   
NAME        GNRC Region GNRC Region        GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region   
Awards          28519.0     29332.0             2404.0      2065.0      2891.0      2070.0      2192.0      2370.0      2357.0      2479.0   

Award Level                                                 Certificate 1+ but < 2 Yr                                                              \
Year               2018        2019        2020        2021                      2010        2011        2012        2013        2014        2015   
NAME        GNRC Region GNRC Region GNRC Region GNRC Region               GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region   
Awards           1644.0      1949.0      2048.0      2251.0                    2551.0      4466.0      5179.0      4114.0      3697.0      3549.0   

Award Level                                                                         Associate's                                                  \
Year               2016        2017        2018        2019        2020        2021        2010        2011        2012        2013        2014   
NAME        GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region   
Awards           3418.0      3304.0      3107.0      2738.0      1824.0      2063.0      3416.0      4153.0      4459.0      4008.0      3644.0   

Award Level                                                                                     Certificate 2+ but < 4 Yr                          \
Year               2015        2016        2017        2018        2019        2020        2021                      2010        2011        2012   
NAME        GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region               GNRC Region GNRC Region GNRC Region   
Awards           3818.0      3749.0      3766.0      4010.0      4160.0      4849.0      5212.0                    1293.0       644.0       177.0   

Award Level                                                                                                              Bachelor's              \
Year               2013        2014        2015        2016        2017        2018        2019        2020        2021        2010        2011   
NAME        GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region   
Awards            118.0       254.0       198.0       253.0       186.0       193.0       234.0       640.0       634.0     10772.0     11215.0   

Award Level                                                                                                                          \
Year               2012        2013        2014        2015        2016        2017        2018        2019        2020        2021   
NAME        GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region GNRC Region   
Awards          11364.0     11904.0     12237.0     12252.0     12501.0     12637.0     12565

In [243]:
#fill nulls with 0.0 and convert to float
data = data.fillna(0.0)
cols = data.columns
data[cols] = data[cols].astype(float)

In [244]:
#loop through all columns grouping certain certificates and higher levels of awards for each geography and year then transpose & reset idnex without dropping
for year in years:
    for geo in geos:
        data[('Certificate < 2 Years', year, '{}'.format(geo))] = data[('Certificate < 1 Yr', year, '{}'.format(geo))] + data[('Certificate 1+ but < 2 Yr',
                                                                                                                                  year, '{}'.format(geo))]
        data[('Certificate > 2 Years', year, '{}'.format(geo))] = data[('Certificate 2+ but < 4 Yr', year, '{}'.format(geo))]
        thelist = [data[("Master's", year, '{}'.format(geo))], data[("Post-Master's", year, '{}'.format(geo))], 
                   data[("Doctorate", year, '{}'.format(geo))]]
        data[("Master's or Above", year, '{}'.format(geo))] = sum(thelist)
        thelist = [data[("Bachelor's", year, '{}'.format(geo))], data[("Post-Baccalaureate", year, '{}'.format(geo))]]
        data[("Bachelor's", year, '{}'.format(geo))] = sum(thelist)
        data = data.drop(columns = [('Certificate < 1 Yr', year, '{}'.format(geo)), ('Certificate 1+ but < 2 Yr', year, '{}'.format(geo)), 
                                      ('Certificate 2+ but < 4 Yr', year, '{}'.format(geo)), ('Post-Baccalaureate', year, '{}'.format(geo))])

In [245]:
data = data.transpose().reset_index(drop = False)

In [246]:
#pivot to take from long to wide format on award levels
data = data.pivot(index = ['Year', 'NAME'], columns = 'Award Level')['Awards']

In [247]:
#reset and remove name from index for joining and feather format
data = data.reset_index(drop = False).rename_axis(None, axis=1).reset_index(drop = True)

In [248]:
#set the index as year and name to remove them from the set of columns for prefix, add the prefix, then drop and keep the indices
data.set_index(['Year', 'NAME'], inplace = True)
data = data.add_prefix('Awards:').reset_index(drop = False)

In [251]:
#calculate percentages
data["Awards %:Associate's"] = percent(data["Awards:Associate's"], data['Awards:All Award Levels'])
data["Awards %:Bachelor's"] = percent(data["Awards:Bachelor's"], data['Awards:All Award Levels'])
data["Awards %:Certificate < 2 Years"] = percent(data["Awards:Certificate < 2 Years"], data['Awards:All Award Levels'])
data["Awards %:Certificate > 2 Years"] = percent(data["Awards:Certificate > 2 Years"], data['Awards:All Award Levels'])
data["Awards %:Doctorate"] = percent(data["Awards:Doctorate"], data['Awards:All Award Levels'])
data["Awards %:Master's"] = percent(data["Awards:Master's"], data['Awards:All Award Levels'])
data["Awards %:Master's or Above"] = percent(data["Awards:Master's or Above"], data['Awards:All Award Levels'])
data["Awards %:Post-Master's"] = percent(data["Awards:Post-Master's"], data['Awards:All Award Levels'])

In [254]:
#final check
awards.head(2)

,Year,NAME,Awards:All Award Levels,Awards:Associate's,Awards:Bachelor's,Awards:Certificate < 2 Years,Awards:Certificate > 2 Years,Awards:Doctorate,Awards:Master's,Awards:Master's or Above,Awards:Post-Master's,Awards %:Associate's,Awards %:Bachelor's,Awards %:Certificate < 2 Years,Awards %:Certificate > 2 Years,Awards %:Doctorate,Awards %:Master's,Awards %:Master's or Above,Awards %:Post-Master's
0,2010,Cheatham County,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010,Davidson County,15864.0,1711.0,5742.0,3221.0,1164.0,958.0,3045.0,4026.0,23.0,10.785426,36.195159,20.303833,7.337368,6.038830,19.194402,25.378215,0.144982
2,2010,Dickson County,294.0,0.0,0.0,248.0,46.0,0.0,0.0,0.0,0.0,0.000000,0.000000,84.353741,15.646259,0.000000,0.000000,0.000000,0.000000
3,2010,GNRC Region,25779.0,3416.0,10823.0,4955.0,1293.0,978.0,4151.0,5292.0,163.0,13.251096,41.983785,19.221071,5.015710,3.793786,16.102254,20.528337,0.632298
4,2010,Houston County,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [252]:
#make a dataframe called awards and ensure the year column is a string to keep dfs straight and join later
awards = data
awards['Year'] = awards['Year'].astype(str)

This retrieves historical and current annual occupational employment disaggregated by high level occupation group.

To retrieve data:
+ Log into JobsEQ and navigate to the "Data Explorer" function  
+ Select the "Occupation" dataset, year/quarter is the only option, we'll have to make it into an annual number  
+ Select parameters: "Region", "Occupation", "Year/Quarter"  
+ Under "Region" select "USA", "Tennessee", GNRC (13), MPO (7), GNRC Region (14), and then under GNRC Region sub all "Counties" and "Places"  
+ For "Year/Quarter" select as many as possible  
+ Under "Occupation" expand "Total-All Occupations" and select the combination of 2 digit SOCS codes and built groups in Jobs EQ as described below, leaving Total as well  

Occupations to Select:  
(SOC Codes)  
+ Total - All Occupations  
+ Healthcare Practitioners and Technical Occupations (29)  
+ Healthcare Support Occupations (31)  
+ Protective Service Occupations (33)  
+ Production Occupations (51)  
+ Transportation and Material Moving Occupations (53)  
(Custom Created Groups)  
+ Business & Financial (11: Management & 13: Business & Financial Operations)  
+ Computer, Mathematical, & Engineering (15: Computer & Mathematical, 17: Architecture & Engineering, & 19: Life, Physical, & Social Science)  
+ Education, Legal, Social, Arts, & Media (21: Community & Social Service, 23: Legal, 25: Educational Instruction & Library, & 27: Art, Design, Entertainment, Sports, & Media)  
+ Food, Property, & Personal Services (35: Food Preparation & Serving, 37: Building & Grounds Cleaning & Maintenance, & 39: Personal Care & Service)  
+ Sales & Office Support (41: Sales & Related & 43: Office & Administrative Support)  
+ Natural Resources, Construction, & Maintenance (45: Farming Fishing & Forestry, 47: Construction & Extraction, & 49: Installation Maintenance & Repair)

+ Make sure Occupation Data is on the top, and Year/Quarter and Region are on the side (this will make the small bit of Excel formatting easier)  
+ Select value: Employment, place of work  

This may exceed the drafting size, in which case you'll have to queue for download. Once it is completed, open and delete the top row that says "Occupation Data", and delete the third row that repeats "Employment (Place of Work)". Rename the FIPS column to GEO_ID, and "Region" to "NAME". Use text to columns to change the GEO_ID column to text, and select all numerical data and format to "general". Save in the "Data Downloads" folder as "JobsEQ_OccupationalEmployment_AllGeos.csv"

In [13]:
#import and examine, dtype str so the GEO_ID formatting doesn't get messed up and so that we can split the time column
data = pd.read_csv('../../Data Downloads/JobsEQ_OccupationalEmployment_AllGeos.csv', dtype = str)
data.head()

,Time,NAME,GEO_ID,Total - All Occupations,Healthcare Practitioners and Technical Occupations (29-0000),Healthcare Support Occupations (31-0000),Protective Service Occupations (33-0000),Production Occupations (51-0000),Transportation and Material Moving Occupations (53-0000),Business & Financial,"Computer, Mathematical, & Engineering","Education, Legal, Social, Arts, & Media","Food, Property, & Personal Services",Sales & Office Support,"Natural Resources, Construction, & Maintenance"
0,2012Q3,USA,0,142247745,7446442.208,5438164.315,3187440.814,9189061.611,11168810.96,14832563.75,6535636.51,14906649.68,20682569.59,35932119.13,12928286.45
1,2012Q3,Tennessee,47,2906688.129,171922.1486,107353.8776,63180.76471,246548.9179,290714.4475,276518.5131,99894.06596,271615.8749,406234.0525,719088.9103,253616.5561
2,2012Q3,MPO,64551,815188.578,49643.31623,30407.00286,19495.41212,54634.99673,69995.10703,82974.5814,29997.06868,84246.65723,113103.4739,215730.2953,64960.66649
3,2012Q3,GNRC,64552,875311.7038,51614.8409,32153.47676,20548.87748,60832.47683,75182.85361,88002.65032,31663.99374,91686.37573,123380.9153,228739.3061,71505.93705
4,2012Q3,GNRC Region,64553,905185.7593,53511.01042,33514.73712,21044.46594,63288.46303,77567.90072,91127.45892,32429.09811,94677.94865,127893.2797,236056.7244,74074.67227


In [7]:
boop = data['Time'].str.split(pat = 'Q', expand = True)
data['Year'] = boop[0]
data['Quarter'] = boop[1]
data.drop(columns = 'Time', inplace = True)

In [12]:
data.head()

Quarter                                                       3                                                                     \
Year                                                       2012                                                                      
NAME                                                        USA    Tennessee          MPO         GNRC  GNRC Region Adams city, TN   
GEO_ID                                                        0           47        64551        64552        64553        4700200   
Total - All Occupations                               142247745  2906688.129   815188.578  875311.7038  905185.7593    52.42284016   
Healthcare Practitioners and Technical Occupati...  7446442.208  171922.1486  49643.31623   51614.8409  53511.01042    0.752052054   
Healthcare Support Occupations (31-0000)            5438164.315  107353.8776  30407.00286  32153.47676  33514.73712    3.075182978   
Protective Service Occupations (33-0000)            3187440.814  63180.76471  19495.41212  20548.87748  21044.46594    1.273752232   

Quarter                                                                                                                             \
Year                                                                                                                                 
NAME                                               Ashland City town, TN Belle Meade city, TN Berry Hill city, TN Bethpage CDP, TN   
GEO_ID                                                           4702180              4704620             4705140          4705500   
Total - All Occupations                                      4048.110799          1160.912425          17747.8372      56.94503222   
Healthcare Practitioners and Technical Occupati...           55.85936208          22.99112142         554.2524422      1.097747052   
Healthcare Support Occupations (31-0000)                     39.27091945          13.75963156         334.0116429      0.180141863   
Protective Service Occupations (33-0000)                     104.1028019          21.92173904         2059.206168      0.802569945   

Quarter                                                                                                                           \
Year                                                                                                                               
NAME                                               Bransford CDP, TN Brentwood city, TN Burns town, TN Castalian Springs CDP, TN   
GEO_ID                                                       4708080            4708280        4709880                   4711440   
Total - All Occupations                                  9.748718518        38960.86827    224.3324408               31.32792056   
Healthcare Practitioners and Technical Occupati...       0.009592506        1861.334794    8.671701983               1.083471554   
Healthcare Support Occupations (31-0000)                  0.01367181        1865.055239    7.528639132               0.642769637   
Protective Service Occupations (33-0000)                 0.010259291        306.7874774    5.330776484                0.08438262   

Quarter                                                                                                                                              \
Year                                                                                                                                                  
NAME                                               Cedar Hill city, TN Charlotte town, TN Christiana CDP, TN Clarksville city, TN Columbia city, TN   
GEO_ID                                                         4711980            4713080            4714860              4715160           4716540   
Total - All Occupations                                    6.111425487        785.7897275        314.3981825          42896.95883       19633.29103   
Healthcare Practitioners and Technical Occ

In [9]:
#we want the last full annual number, which is 2021 at this time, so knock out the handful of quarters in 2012 and 2022
data = data.loc[(data['Year'] != 2012)&(data['Year'] != 2022)]

In [10]:
#to average out years, make a list of years and of geographies in the dataframe
geos = list(data.Year.unique())
geos = list(data.NAME.unique())

In [11]:
data = data.set_index(['Quarter', 'Year', 'NAME']).transpose()

In [ ]:
cols = data.columns
for year in years:
    for geo in geos:
        thelist = [data[('Q1', year, '{}'.format(geo))], data[('Q2', year, '{}'.format(geo))], data[('Q3', year, '{}'.format(geo))], 
                  data[('Q4', year, '{}'.format(geo))]]
        data[('All', year, '{}'.format(geo))] = (sum(thelist))/4
data = data.drop(columns = cols)

## Join all JobsEQ Annual data

In [225]:
dfs = [ue, gdp, awards]
df = reduce(lambda  left,right: pd.merge(left,right,on=['NAME', 'Year'], how='outer'), dfs)

In [227]:
df.tail()

,NAME,Year,LF:Unemployment %,GDP:GDP,Awards:All Award Levels,Awards:Associate's,Awards:Bachelor's,Awards:Certificate < 2 Years,Awards:Certificate > 2 Years,Awards:Doctorate,Awards:Master's,Awards:Master's or Above,Awards:Post-Master's,Awards %:Associate's,Awards %:Bachelor's,Awards %:Certificate < 2 Years,Awards %:Certificate > 2 Years,Awards %:Doctorate,Awards %:Master's,Awards %:Master's or Above,Awards %:Post-Master's
432,Stewart County,2022,3.764563,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
433,Sumner County,2022,2.661477,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
434,Trousdale County,2022,2.896448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
435,Williamson County,2022,2.265584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
436,Wilson County,2022,2.559500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [255]:
df.to_feather('../../Outputs/JobsEQ_AnnualData')